In [ ]:
%load_ext autoreload
%autoreload 2

[lstm](https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv")

In [ ]:
df = pd.read_csv("shampoo.csv")
df.head()

,Month,Sales
0,1-01,266.0
1,1-02,145.9
2,1-03,183.1
3,1-04,119.3
4,1-05,180.3
5,1-06,168.5
6,1-07,231.8
7,1-08,224.5
8,1-09,192.8
9,1-10,122.9


In [ ]:
# df.to_csv("shampoo.csv", index=False)

In [9]:
# load and plot dataset
from pandas import read_csv
import datetime
from matplotlib import pyplot
# load dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
series = read_csv('shampoo.csv', header=0, parse_dates=[0], index_col=0, date_parser=parser)
# summarize first few rows
print(series.head())
# line plot
series.plot()
pyplot.show()

/var/folders/rs/82njg9p15j9ccf6dc3sxp0500000gn/T/ipykernel_2409/363522217.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  series = read_csv('shampoo.csv', header=0, parse_dates=[0], index_col=0, date_parser=parser)


AttributeError: module 'datetime' has no attribute 'strptime'